In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_memory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [3]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [16]:
# 1. 프롬프트 자리에 히스토리 파트 확보
system_prompt = """
당신은 이력서, 포트폴리오, 자소서를 잘 쓸수있게 도와주는 15년차 전문가 커리어 코칭 강사입니다.
회사 면접관이 이력서, 포트폴리오, 자소서를 봤을 때 합격을 시키고 싶다라는 느낌을 가지게 해줬으면 좋겠어
하단 [FORMAT]에 맞게 답변 부탁드립니다.

[FORMAT]
1. 이모지 사용 금지
2. 반복적인 단어와 문장을 사용하면 안 됩니다.
3. 직관적이며 보기 쉬운 느낌을 줬으면 좋겠습니다.
"""

user_prompt = """
저희는 처음에 YOLO11m모델 Object ditection을 활용한 태양광 패널 이상 객체 탐지를 위해 roboflow 사이트에서 3개의 태양광 패널 데이터셋을 확보하여 데이터가 부족한 클래스는 삭제하고 비슷한 클래스끼리 합쳐주는 방법으로 10,130개의 데이터셋을 확보하였고 확보한 데이터는 클래스별로 바운딩박스가 되어 있었으며 이 데이터셋을 가지고 전이학습을 진행하였고 생각보다 성능이 잘 나오지 않아서 직접 수작업으로 데이터셋을 바운딩박스 작업을 했습니다. 아쉬웠던 점은 클래스별로 모양과 위치가 가지각색인데 사각형 박스로만 객체를 표시 할 수 있기 때문에 섬세한 작업을 할 수 없었습니다. 5천개 정도를 직접 바운딩박스 작업을 한 뒤 학습을 진행하였지만 오히려 학습결과가 안 좋게 나왔습니다. 그래서 저희가 생각한 대안은 YOLO Segmentaion을 활용하여 태양광 패널의 문제가 되는 부분을 클래스별 객체의 외곽선을 따라 픽셀 단위로 정확히 분리한 새로운 3,343개의 외부충격 데이터셋을 가지고 Base Fine-tuning을 진행하였습니다. 왜냐하면 사용하려는 yolo11m-seg.pt 모델은 태양광 패널에 대한 지식이 없기 때문에 새로운 지식을 학습 시켜서 성능을 높이고 이전에 학습 된 best.pt는 검증 기반으로 가장 성능이 좋았던 가중치를 불러와서 학습을 시키면 학습 결과에 미치는 영향은 "빠른 수렴", "데이터가 적을 때의 일반화 향상", "도메인 특화", "학습 실패 극복/ 재현성", 연속 연구/모델 개선 등의 효과가 있다는 것을 알게 되었고 실제 학습결과와 테스트를 해보았을 때 좋은 결과가 나왔습니다. 추가로 태양광 패널 내부충격 데이터셋을 수집하였고 내부충격은 핫스팟이라는 열화상 카메라로 촬영한 데이터이며 특정 지점의 온도가 주변보다 비정상적으로 높은 부분을 표시가 되어있는 데이터이며
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

chain = prompt | model | StrOutputParser()

In [ ]:
# 2. 세션(사용자)별 대화 저장소 만들기

# 각 사용자의 대화 내용을 따로 저장하기 위한 딕셔너리
# 예: {"user1": 대화기록1, "user2": 대화기록2}
stores: Dict[str, InMemoryChatMessageHistory] = {}

# K : 한 사용자당 저장할 최대 대화 개수 (최근 6개까지만 기억)
K = 6

def get_stores(session_id: str):
    # 만약 이 사용자가 처음 대화하는 경우라면,
    # 새 대화 기록(InMemoryChatMessageHistory)을 만들어서 저장
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()

    # 해당 사용자의 대화 기록을 가져오기
    hist = stores.setdefault(session_id, InMemoryChatMessageHistory())

    # 대화가 6개(K)보다 많아지면 → 최근 6개만 남기고 나머지는 삭제
    if len(hist.messages) > K:
        hist.messages[:] = hist.messages[-K:]

    # 해당 사용자의 대화 기록(hist)을 반환
    return hist


In [25]:
# 3. 히스토리와 연결
with_history = RunnableWithMessageHistory(
    chain, 
    get_stores,
    input_messages_key="question",
    history_messages_key="history"
)

In [19]:
config = {"configurable" : {"session_id" : "user-123"}}
result = with_history.invoke({"question":user_prompt}, config=config)
print(result)

[이력서 및 자기소개서 작성 가이드]

프로젝트명: 태양광 패널 이상 객체 탐지 및 세그멘테이션 기반 성능 개선

1. 프로젝트 개요  
- YOLO11m 모델을 활용해 태양광 패널의 이상 객체 탐지 수행  
- Roboflow에서 3개 데이터셋 통합, 클래스 통합 및 불필요 클래스 삭제로 10,130개 데이터 확보  
- 바운딩박스 기반 전이학습 진행 후 성능 한계 경험  

2. 문제 인식 및 개선 시도  
- 클래스별 객체 모양과 위치 다양성으로 인해 사각형 바운딩박스 한계 확인  
- 직접 5,000여 개 바운딩박스 수작업 보완 시도, 오히려 성능 저하 발생  

3. 대안 및 성과  
- YOLO Segmentation 기법 도입, 픽셀 단위 외곽선 분리로 정밀한 객체 분할 가능  
- 3,343개 외부충격 데이터셋 구축 후 Base Fine-tuning 수행  
- yolo11m-seg.pt 모델에 태양광 패널 특화 지식 학습, best.pt 가중치 활용으로 빠른 수렴 및 일반화 향상 달성  
- 학습 실패 극복 및 도메인 특화 모델 개발에 성공, 테스트 결과 우수한 성능 확인  

4. 추가 데이터 수집 및 활용  
- 내부충격 데이터셋 확보 (핫스팟 열화상 카메라 활용)  
- 비정상 고온 부위 탐지 데이터로 모델 성능 다각도 검증 및 보완 예정  

[작성 팁]  
- 핵심 성과와 문제 해결 과정을 명확히 구분해 기술  
- 수치와 구체적 데이터(데이터셋 크기, 모델명 등)를 포함해 신뢰도 강화  
- 기술적 한계와 개선 방향을 솔직하게 서술해 전문성 강조  
- 결과에 대한 객관적 평가(성능 향상, 테스트 결과 등)를 반드시 포함  

이 내용을 바탕으로 이력서 및 자기소개서에 간결하고 체계적으로 작성하면 면접관에게 전문성과 문제 해결 능력을 효과적으로 어필할 수 있습니다.


In [26]:
config = {"configurable" : {"session_id" : "user-123"}}
result = with_history.invoke({"question":user_prompt}, config=config)
print(result)
stores["user-123"].messages

[이력서 및 자기소개서 작성 가이드]

프로젝트명: 태양광 패널 이상 객체 탐지 및 세그멘테이션 기반 성능 향상

1. 프로젝트 개요  
- YOLO11m 모델을 활용해 태양광 패널 이상 객체 탐지 수행  
- Roboflow에서 3개 데이터셋 확보 후 클래스 통합 및 불필요 클래스 삭제로 10,130개 데이터셋 구축  
- 바운딩박스 기반 전이학습 진행, 초기 성능 미흡으로 직접 5,000개 수작업 바운딩박스 재작업 시도  

2. 문제점 및 한계  
- 객체 모양과 위치 다양성에 비해 사각형 바운딩박스 한계로 섬세한 객체 표현 어려움  
- 수작업 데이터셋 추가 후 오히려 학습 성능 저하 발생  

3. 해결 방안 및 성과  
- YOLO Segmentation 기법 도입, 픽셀 단위 외곽선 분리로 3,343개 외부충격 데이터셋 구축  
- Base Fine-tuning 적용, yolo11m-seg.pt 모델에 태양광 패널 특화 지식 학습  
- 이전 best.pt 가중치 활용해 빠른 수렴 및 일반화 성능 향상 달성  
- 학습 실패 극복 및 도메인 특화 모델 완성, 테스트 결과 우수한 성능 확인  

4. 추가 연구 및 데이터 수집  
- 태양광 패널 내부충격 탐지 위해 핫스팟 열화상 카메라 데이터 확보  
- 비정상 고온 영역을 정확히 표시하는 내부충격 데이터셋 구축 중  

[작성 팁]  
- 핵심 내용은 간결하고 명확하게 기술하세요.  
- 기술적 어려움과 해결 과정을 구체적으로 서술해 문제 해결 능력을 강조하세요.  
- 성과는 수치나 결과 중심으로 표현해 신뢰도를 높이세요.  
- 전문 용어는 필요 시 간단히 설명해 이해도를 높이세요.  

이 내용을 바탕으로 이력서 경력사항이나 자기소개서 프로젝트 경험란에 다음과 같이 작성할 수 있습니다.

---

**태양광 패널 이상 객체 탐지 및 세그멘테이션 모델 개발**  
- YOLO11m 기반 전이학습으로 10,130개 데이터셋 구축 및 초기 탐지 모델 개발  
- 바운딩박스 한계 극복 위해 YOLO

[HumanMessage(content='\n저희는 처음에 YOLO11m모델 Object ditection을 활용한 태양광 패널 이상 객체 탐지를 위해 roboflow 사이트에서 3개의 태양광 패널 데이터셋을 확보하여 데이터가 부족한 클래스는 삭제하고 비슷한 클래스끼리 합쳐주는 방법으로 10,130개의 데이터셋을 확보하였고 확보한 데이터는 클래스별로 바운딩박스가 되어 있었으며 이 데이터셋을 가지고 전이학습을 진행하였고 생각보다 성능이 잘 나오지 않아서 직접 수작업으로 데이터셋을 바운딩박스 작업을 했습니다. 아쉬웠던 점은 클래스별로 모양과 위치가 가지각색인데 사각형 박스로만 객체를 표시 할 수 있기 때문에 섬세한 작업을 할 수 없었습니다. 5천개 정도를 직접 바운딩박스 작업을 한 뒤 학습을 진행하였지만 오히려 학습결과가 안 좋게 나왔습니다. 그래서 저희가 생각한 대안은 YOLO Segmentaion을 활용하여 태양광 패널의 문제가 되는 부분을 클래스별 객체의 외곽선을 따라 픽셀 단위로 정확히 분리한 새로운 3,343개의 외부충격 데이터셋을 가지고 Base Fine-tuning을 진행하였습니다. 왜냐하면 사용하려는 yolo11m-seg.pt 모델은 태양광 패널에 대한 지식이 없기 때문에 새로운 지식을 학습 시켜서 성능을 높이고 이전에 학습 된 best.pt는 검증 기반으로 가장 성능이 좋았던 가중치를 불러와서 학습을 시키면 학습 결과에 미치는 영향은 "빠른 수렴", "데이터가 적을 때의 일반화 향상", "도메인 특화", "학습 실패 극복/ 재현성", 연속 연구/모델 개선 등의 효과가 있다는 것을 알게 되었고 실제 학습결과와 테스트를 해보았을 때 좋은 결과가 나왔습니다. 추가로 태양광 패널 내부충격 데이터셋을 수집하였고 내부충격은 핫스팟이라는 열화상 카메라로 촬영한 데이터이며 특정 지점의 온도가 주변보다 비정상적으로 높은 부분을 표시가 되어있는 데이터이며\n', additional_kwargs={}, response_metadata={}),
 AIMessage(co

In [27]:
config = {"configurable" : {"session_id" : "user-123"}}
result = with_history.invoke({"question":user_prompt}, config=config)
print(result)
stores["user-123"].messages

[이력서 및 자기소개서 작성 예시]

프로젝트명: 태양광 패널 이상 객체 탐지 및 세그멘테이션 모델 개발

1. 프로젝트 개요  
- YOLO11m Object Detection 모델을 활용해 태양광 패널 이상 객체 탐지 수행  
- Roboflow에서 3개 데이터셋 확보 후, 데이터 부족 클래스 삭제 및 유사 클래스 통합으로 10,130개 데이터셋 구축  
- 클래스별 바운딩박스 라벨링 데이터 기반 전이학습 진행  

2. 문제점 및 대응  
- 초기 전이학습 성능 저조로 5,000개 데이터셋 직접 바운딩박스 재라벨링 수행  
- 사각형 바운딩박스 한계로 객체 모양과 위치 다양성 반영 어려움  
- 재학습 결과 성능 개선 미흡  

3. 해결책 및 성과  
- YOLO Segmentation 기법 도입, 픽셀 단위 외곽선 분할로 3,343개 외부충격 데이터셋 구축  
- Base Fine-tuning 적용, yolo11m-seg.pt 모델에 태양광 패널 특화 지식 학습  
- 이전 best.pt 가중치 활용해 빠른 수렴, 일반화 성능 향상, 도메인 특화 모델 완성  
- 학습 실패 극복 및 재현성 확보, 테스트에서 우수한 성능 확인  

4. 추가 연구  
- 핫스팟 열화상 카메라로 촬영한 내부충격 데이터셋 수집 및 분석 진행  
- 비정상 고온 영역 탐지로 내부 결함 진단 정확도 향상 기대  

[작성 팁]  
- 핵심 내용은 간결하고 명확하게 기술  
- 문제 인식과 해결 과정을 구체적으로 서술해 문제 해결 능력 강조  
- 성과는 수치 및 결과 중심으로 표현해 신뢰도 강화  
- 전문 용어는 간단히 설명해 이해도 증진  

---

**경력사항 예시**  
태양광 패널 이상 객체 탐지 및 세그멘테이션 모델 개발  
- YOLO11m 기반 10,130개 데이터셋 구축 및 전이학습 수행  
- 바운딩박스 한계 극복 위해 YOLO Segmentation 도입, 3,343개 외부충격 데이터셋으로 픽셀 단위 분할  
- Base Fine-tuning 및 best

[HumanMessage(content='\n저희는 처음에 YOLO11m모델 Object ditection을 활용한 태양광 패널 이상 객체 탐지를 위해 roboflow 사이트에서 3개의 태양광 패널 데이터셋을 확보하여 데이터가 부족한 클래스는 삭제하고 비슷한 클래스끼리 합쳐주는 방법으로 10,130개의 데이터셋을 확보하였고 확보한 데이터는 클래스별로 바운딩박스가 되어 있었으며 이 데이터셋을 가지고 전이학습을 진행하였고 생각보다 성능이 잘 나오지 않아서 직접 수작업으로 데이터셋을 바운딩박스 작업을 했습니다. 아쉬웠던 점은 클래스별로 모양과 위치가 가지각색인데 사각형 박스로만 객체를 표시 할 수 있기 때문에 섬세한 작업을 할 수 없었습니다. 5천개 정도를 직접 바운딩박스 작업을 한 뒤 학습을 진행하였지만 오히려 학습결과가 안 좋게 나왔습니다. 그래서 저희가 생각한 대안은 YOLO Segmentaion을 활용하여 태양광 패널의 문제가 되는 부분을 클래스별 객체의 외곽선을 따라 픽셀 단위로 정확히 분리한 새로운 3,343개의 외부충격 데이터셋을 가지고 Base Fine-tuning을 진행하였습니다. 왜냐하면 사용하려는 yolo11m-seg.pt 모델은 태양광 패널에 대한 지식이 없기 때문에 새로운 지식을 학습 시켜서 성능을 높이고 이전에 학습 된 best.pt는 검증 기반으로 가장 성능이 좋았던 가중치를 불러와서 학습을 시키면 학습 결과에 미치는 영향은 "빠른 수렴", "데이터가 적을 때의 일반화 향상", "도메인 특화", "학습 실패 극복/ 재현성", 연속 연구/모델 개선 등의 효과가 있다는 것을 알게 되었고 실제 학습결과와 테스트를 해보았을 때 좋은 결과가 나왔습니다. 추가로 태양광 패널 내부충격 데이터셋을 수집하였고 내부충격은 핫스팟이라는 열화상 카메라로 촬영한 데이터이며 특정 지점의 온도가 주변보다 비정상적으로 높은 부분을 표시가 되어있는 데이터이며\n', additional_kwargs={}, response_metadata={}),
 AIMessage(co

In [21]:
stores["user-123"].messages

[HumanMessage(content='\n저희는 처음에 YOLO11m모델 Object ditection을 활용한 태양광 패널 이상 객체 탐지를 위해 roboflow 사이트에서 3개의 태양광 패널 데이터셋을 확보하여 데이터가 부족한 클래스는 삭제하고 비슷한 클래스끼리 합쳐주는 방법으로 10,130개의 데이터셋을 확보하였고 확보한 데이터는 클래스별로 바운딩박스가 되어 있었으며 이 데이터셋을 가지고 전이학습을 진행하였고 생각보다 성능이 잘 나오지 않아서 직접 수작업으로 데이터셋을 바운딩박스 작업을 했습니다. 아쉬웠던 점은 클래스별로 모양과 위치가 가지각색인데 사각형 박스로만 객체를 표시 할 수 있기 때문에 섬세한 작업을 할 수 없었습니다. 5천개 정도를 직접 바운딩박스 작업을 한 뒤 학습을 진행하였지만 오히려 학습결과가 안 좋게 나왔습니다. 그래서 저희가 생각한 대안은 YOLO Segmentaion을 활용하여 태양광 패널의 문제가 되는 부분을 클래스별 객체의 외곽선을 따라 픽셀 단위로 정확히 분리한 새로운 3,343개의 외부충격 데이터셋을 가지고 Base Fine-tuning을 진행하였습니다. 왜냐하면 사용하려는 yolo11m-seg.pt 모델은 태양광 패널에 대한 지식이 없기 때문에 새로운 지식을 학습 시켜서 성능을 높이고 이전에 학습 된 best.pt는 검증 기반으로 가장 성능이 좋았던 가중치를 불러와서 학습을 시키면 학습 결과에 미치는 영향은 "빠른 수렴", "데이터가 적을 때의 일반화 향상", "도메인 특화", "학습 실패 극복/ 재현성", 연속 연구/모델 개선 등의 효과가 있다는 것을 알게 되었고 실제 학습결과와 테스트를 해보았을 때 좋은 결과가 나왔습니다. 추가로 태양광 패널 내부충격 데이터셋을 수집하였고 내부충격은 핫스팟이라는 열화상 카메라로 촬영한 데이터이며 특정 지점의 온도가 주변보다 비정상적으로 높은 부분을 표시가 되어있는 데이터이며\n', additional_kwargs={}, response_metadata={}),
 AIMessage(co